In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import xarray as xr

import esmtools as et
from esmtools import stats

import shapefile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gp
import regionmask
import matplotlib.patches as mpatches

In [2]:
# Open dataset 
dir = '/chinook/dallmann/SOARS/'
fn = 'tos_Omon_CESM2_omip2_r1i1p1f1_gr_030601-036612.nc'

data_tos = xr.open_dataset(dir+fn)
data_tos

/opt/miniconda3/envs/prod/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'tos' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:    (d2: 2, lat: 180, lon: 360, time: 732)
Coordinates:
  * time       (time) object 0306-01-15 12:00:00 ... 0366-12-15 12:00:00
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: d2
Data variables:
    time_bnds  (time, d2) object ...
    lat_bnds   (lat, d2) float64 ...
    lon_bnds   (lon, d2) float64 ...
    tos        (time, lat, lon) float32 ...
Attributes: (12/46)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            OMIP
    branch_method:          no parent
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    case_id:                1540
    ...                     ...
    tracking_id:            hdl:21.14100/035f5fb2-1e7b-46a3-a72f-268f1f6ea194
    variable_id:            tos
    variant_info:           CMIP6 omip2 experiment (JRA forcing) with interac...
    variant_label:          r1i1p1f1
    source_type:            OGCM BGC
    comment:                Experiment run for 6 cycles of 1958-2018 JRA55 fo...

In [3]:
# Change the time to be from DatetimeNoLeap to a Datetime64
data_tos['time'] = np.arange('1958-01-01','2019-01-01',dtype = 'datetime64[M]')
data_tos

<xarray.Dataset>
Dimensions:    (d2: 2, lat: 180, lon: 360, time: 732)
Coordinates:
  * time       (time) datetime64[ns] 1958-01-01 1958-02-01 ... 2018-12-01
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: d2
Data variables:
    time_bnds  (time, d2) object 0306-01-01 00:00:00 ... 0367-01-01 00:00:00
    lat_bnds   (lat, d2) float64 -90.0 -89.0 -89.0 -88.0 ... 88.0 89.0 89.0 90.0
    lon_bnds   (lon, d2) float64 0.0 1.0 1.0 2.0 2.0 ... 358.0 359.0 359.0 360.0
    tos        (time, lat, lon) float32 ...
Attributes: (12/46)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            OMIP
    branch_method:          no parent
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    case_id:                1540
    ...                     ...
    tracking_id:            hdl:21.14100/035f5fb2-1e7b-46a3-a72f-268f1f6ea194
    variable_id:            tos
    variant_info:           CMIP6 omip2 experiment (JRA forcing) with interac...
    variant_label:          r1i1p1f1
    source_type:            OGCM BGC
    comment:                Experiment run for 6 cycles of 1958-2018 JRA55 fo...

In [4]:
# select dimensions!
data_tos['time'] = pd.date_range("1958-01", "2018-12", freq="MS")
tos = data_tos['tos']
tos = tos.sel(lon=slice(190,270),lat=slice(-10,70))

In [10]:
# remove long term trend for each grid cell
var = stats.rm_poly(tos,order=2,dim='time',nan_policy='drop')

# 1982-2010 period
base = var.sel(time=slice('1982-01','2010-12'))
tos_base = base

# Blob period
#blob = var.sel(time=slice('2013-07','2016-06'))
#tos_blob = blob

# Difference between the two periods
#blob_anom = blob.groupby('time.month') - base.groupby('time.month').mean()

# Resample to two month resolution
#fosi = blob_anom.resample(time='2M').mean()

<xarray.DataArray (lat: 80, lon: 80, time: 348)>
array([[[ 0.06384714,  0.25971397,  0.23060959, ..., -0.54175146,
         -0.50153979, -0.56201098],
        [ 0.06301214,  0.18894965,  0.19161103, ..., -0.58170998,
         -0.54129101, -0.58473208],
        [ 0.07634902,  0.13170322,  0.15266883, ..., -0.61536368,
         -0.57379868, -0.60098704],
        ...,
        [ 0.34274836,  1.31485846,  1.89065861, ..., -2.76007667,
         -2.49326905, -1.90033003],
        [ 0.42639274,  1.41977654,  1.96795903, ..., -2.88911567,
         -2.58059079, -1.90236882],
        [ 0.5456311 ,  1.53152355,  2.0427057 , ..., -3.03048991,
         -2.67697182, -1.88619328]],

       [[-0.05364711,  0.09380252,  0.10750548, ..., -0.6674194 ,
         -0.72012971, -0.84220885],
        [-0.04566177,  0.0511122 ,  0.07761698, ..., -0.71154842,
         -0.76171227, -0.87193191],
        [-0.0135131 ,  0.01618794,  0.05453209, ..., -0.74772223,
         -0.79966729, -0.90259316],
...
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[-1.97553283, -1.9880196 , -1.99853401, ...,  3.24561908,
         -0.83063956, -2.46193858],
        [-2.12059886, -2.13079686, -2.14192236, ...,  3.09049352,
         -1.31655123, -2.61458996],
        [-2.20017861, -2.20880077, -2.22101895, ...,  2.90791237,
         -1.69964162, -2.70201699],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [-0.56882535, -0.58107716, -0.59123232, ...,  0.96439815,
         -0.55248142, -0.70533311]]])
Coordinates:
  * time     (time) datetime64[ns] 1982-01-01 1982-02-01 ... 2010-12-01
  * lat      (lat) float64 -9.5 -8.5 -7.5 -6.5 -5.5 ... 65.5 66.5 67.5 68.5 69.5
  * lon      (lon) float64 190.5 191.5 192.5 193.5 ... 266.5 267.5 268.5 269.5

In [8]:
def panels(data,description,variable,depth,lower_bound,upper_bound, step,cmap):
    f, axs = plt.subplots(nrows=3,ncols=6,figsize=(14,4),
                          subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180))) 
    axs = axs.flatten()
    for i in range(0,18):
        im = axs[i].contourf(fosi.lon, fosi.lat, fosi.isel(time=i),
                         transform=ccrs.PlateCarree(), 
                         cmap=cmap,
                         levels=np.arange(lower_bound, upper_bound, step),
                         extend='both')
        axs[i].add_feature(cfeature.LAND, color='k')
        axs[i].add_patch(mpatches.Rectangle(xy=[210, 40], width=15, height=10,
                                    edgecolor='k',
                                    facecolor='none',
                                    transform=ccrs.PlateCarree())
                 )
    plt.suptitle("FOSI: " + description + " " + variable + " " + depth + " (de-trended)")
    axs[0].set_title("Jul-Aug"); axs[1].set_title("Sept-Oct")
    axs[2].set_title("Nov-Dec"); axs[3].set_title("Jan-Feb")
    axs[4].set_title("Mar-Apr"); axs[5].set_title("May-Jun")
    plt.text(-790,215,'2013-14')
    plt.text(-790,120,'2014-15')
    plt.text(-790,25,'2015-16')
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes([0.82, 0.15, 0.05, 0.7])
    f.colorbar(im, cax=cbar_ax,fraction=0.046,pad=0.04)
    
